Library Imports

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 5.0 MB/s 
     |████████████████████████████████| 596 kB 34.4 MB/s 
     |████████████████████████████████| 59 kB 6.9 MB/s 
     |████████████████████████████████| 3.3 MB 43.6 MB/s 
     |████████████████████████████████| 895 kB 25.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import numpy as np
import re

Dataset

In [ ]:
pdt = pd.read_csv('train (1).csv')

In [ ]:
pdt.head()

,id,query,product_title,product_description,median_relevance,relevance_variance
0,1,bridal shower decorations,Accent Pillow with Heart Design - Red/Black,Red satin accent pillow embroidered with a hea...,1,0.000
1,2,led christmas lights,Set of 10 Battery Operated Multi LED Train Chr...,Set of 10 Battery Operated Train Christmas Lig...,4,0.000
2,4,projector,ViewSonic Pro8200 DLP Multimedia Projector,NaN,4,0.471
3,5,wine rack,Concept Housewares WR-44526 Solid-Wood Ceiling...,"Like a silent and sturdy tree, the Southern En...",4,0.000
4,7,light bulb,Wintergreen Lighting Christmas LED Light Bulb ...,"WTGR1011\nFeatures\nNickel base, 60,000 averag...",2,0.471


In [ ]:
pdt.sort_values(by='query',inplace=True)

In [ ]:
pdt.head()

,id,query,product_title,product_description,median_relevance,relevance_variance
3291,10557,16 gb memory card,Sandisk 32GB Dual-adapter Micro SD Card,"SanDisk® 32 GB Micro SD card offers an easy, a...",2,0.471
9274,29818,16 gb memory card,SanDisk Ultra - flash memory card - 16 GB - SD...,Get the highe video recording performance with...,4,0.000
152,472,16 gb memory card,Sandisk 16GB non-HS MSD Flash Drive 3in1 - Bla...,"SanDisk Elevate 16GB non-HS MSD 3in, microSD m...",4,0.000
4276,13823,16 gb memory card,Lot of 2x 16GB = 32GB SanDisk SDHC Flash SD Me...,eBay item number:381062947092\n\n\n\tSeller as...,4,0.000
7494,24159,16 gb memory card,Kingston - flash memory card - 16 GB - microSDHC,The microSDHC cards offer higher storage capac...,4,0.000


In [ ]:
product_names=list(pdt['product_title'].unique())

In [ ]:
len(product_names)

9708

Preprocessing

In [ ]:
import nltk
from nltk.util import ngrams
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import torch
from transformers import BertTokenizer, BertModel

In [ ]:
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

BERT model for creating product and query embbedding

In [ ]:
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True,
                                  )
model.eval()

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [ ]:
def bert_embedding(names):
  emd_lst = []

  for i in range(len(names)):
    #print(data[i])
    #data cleaning and tokenization
    title = names[i].lower()
    title=title.replace('-'," ")
    data = " [CLS] " + title + " [SEP] "
    tokens = tokenizer.tokenize(data)

    #index values for tokens
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokens)

    #segment IDs
    seg_ids = [1 for j in range(len(tokens))]

    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([seg_ids])

    with torch.no_grad():

      outputs = model(tokens_tensor, segments_tensors)
      hidden_states = outputs[2]

    #token vectors
    token_vecs = hidden_states[-2][0]

    #product vectors
    emd = torch.mean(token_vecs, dim=0)
    emd_lst.append(emd)

  return emd_lst

Neural Network Architecture

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import AveragePooling1D,BatchNormalization,Activation

In [ ]:
nn = tf.keras.Sequential()
nn.add(AveragePooling1D(pool_size=2, strides=None, padding="valid"))
nn.add(BatchNormalization())
nn.add(Activation('tanh'))

In [ ]:
def create_embeddings(nn, data):
  ogtensor = bert_embedding(data)
  #print(len(ogtensor))
  res=[]
  for i in range(len(ogtensor)):
    x = tf.reshape(ogtensor[i], [1, len(ogtensor[i]), 1])
    tmp = nn(x)
    res.append(tf.reshape(tmp,len(tmp[0])))
  return res

In [ ]:
len(product_names)

9708

Product Embeddings

In [ ]:
product_embeddings = create_embeddings(nn, product_names)

In [ ]:
product_embeddings[0]

In [ ]:
len(product_embeddings)

9708

Query Embedding

In [ ]:
query="Memory Card 16gb"

In [ ]:
q_emd = create_embeddings(nn, [query])

In [ ]:
q_emd

Finding Similarity between query and products

In [ ]:
def cosine_similarity(x, y):
  return abs(tf.get_static_value(tf.keras.losses.cosine_similarity(x, y)))

In [ ]:
dist=dict()
for i in range(len(product_embeddings)):
  val=cosine_similarity(product_embeddings[i], q_emd)[0]
  if val >= 0.8:
    dist[i]=val

In [ ]:
len(dist)

35

Ranking based on cosine similarity between query and products

In [ ]:
product_id=sorted(dist.items(), key =lambda kv:(kv[1], kv[0]),reverse=True)

In [ ]:
pred_results = []
for i in range(len(product_id)):
  val=product_id[i][0]
  pred_results.append(product_names[val])
  print(product_names[val])

Kingston - flash memory card - 16 GB - microSDHC
Verbatim - flash memory card - 16 GB - SDHC
Toshiba - flash memory card - 16 GB - microSDHC
SanDisk - flash memory card - 16 GB - microSDHC
EP Memory - flash memory card - 16 GB - CompactFlash
SanDisk - flash memory card - 16 GB - SDHC
Cisco - flash memory card - 16 GB - SD
SanDisk - flash memory card - 8 GB - microSDHC
Lexar 100x 8GB SDHC Memory Card - Black
Memorex TravelCard - flash memory card - 32 GB - SDHC
Toshiba 16GB microSD Memory Card with Standard Adapter
Kingston Ultimate - flash memory card - 16 GB - CompactFlash
Eye-Fi Pro X2 - wireless memory card - 16 GB - SDHC
Lexar 16GB Full HD-Video SDHC Memory Card
Wintec FileMate 16GB Professional SDHC Flash Memory Card Class 10
Micro SD 4GB 8GB 16GB TF Flash Memory Card SDHC MicroSD with adapter
Sandisk 32GB Dual-adapter Micro SD Card
16GB MicroSD Memory Card with SD Adapter (Pack of 2)
Kingston Multi-Kit / Mobility Kit - flash memory card - 16 GB - microSDHC
Sandisk 16gb Micro SD M

Embeddings for given Queries

In [ ]:
pre_queries=list(pdt['query'].unique())

In [ ]:
pre_queries[:5]

['16 gb memory card',
 '8 ounce mason jars',
 'acoustic guitar clamp',
 'adidas fragance',
 'adidas pants']

In [ ]:
query_emds = create_embeddings(nn, pre_queries)

In [ ]:
query_similarity = dict()

for i in range(len(query_emds)):
  query_similarity[pre_queries[i]] = cosine_similarity(query_emds[i], q_emd)[0]

In [ ]:
org_qry = sorted(query_similarity.items(), key =lambda kv:(kv[1], kv[0]),reverse=True)[0][0]

In [ ]:
org_qry

'16 gb memory card'

In [ ]:
actual_results = pdt[pdt['query'] == org_qry]['product_title']

In [ ]:
rel_retrieved = set(actual_results).intersection(set(pred_results))

precision = len(rel_retrieved) / len(pred_results)
recall = len(rel_retrieved) / len(actual_results)
fscore = (2 * precision * recall)/(precision+recall)

print('Precision: ', precision, '\nRecall: ', recall, '\nF-measure: ', fscore)

Precision:  0.8 
Recall:  0.4375 
F-measure:  0.5656565656565656


In [ ]:
len(rel_retrieved)

52

In [ ]:
len(pred_results)

89

In [ ]:
len(actual_results)

64

In [ ]:
actual_results

In [ ]:
rank_pred = [i for i in range(len(rel_retrieved), 0, -1)]
rank_actual = [i for i in range(len(actual_results), 0, -1)]

In [ ]:
combined = []
rel_retrieved=list(rel_retrieved)
actual_results=list(actual_results)
for i in range(len(rank_pred)):
  id = actual_results.index(rel_retrieved[i])
  combined.append((rank_pred[i], rank_actual[id]))

In [ ]:
relevance=sorted(combined, key=lambda x: x[1],reverse=True)

In [ ]:
import numpy as np
DCG = 0
for i in range(len(rel_retrieved)):
  DCG += relevance[i][0]/(np.log10(i+2)/np.log10(2))

In [ ]:
DCG

117.41530238945502

In [ ]:
iDCG = 0
for i in range(len(rel_retrieved)):
  iDCG += (len(rel_retrieved)-i)/(np.log10(i+2)/np.log10(2))
iDCG

153.7479918201538

In [ ]:
nDCG=DCG/iDCG
nDCG

0.7636867382749375

Embedding from file



In [ ]:
f=open("/content/drive/MyDrive/Copy of embeddings.txt")

In [ ]:
prod_embedding=f.readlines()

In [ ]:
prod_embedding

In [ ]:
for i in range(len(prod_embedding)):
  prod_embedding[i]=prod_embedding[i].replace("\n","")
  prod_embedding[i]=prod_embedding[i].lstrip()

In [ ]:
ogvec=[]
vec=[]
for i in range(len(prod_embedding)):
    if '),' in prod_embedding[i]:
      val=prod_embedding[i].split('),')
      val[0]=val[0].replace(']',"")
      vec.append(val[0])
      ogvec.append(vec)
      val[1]=val[1].replace('tensor([',"")
      val[1]=val[1].replace(']',"")
      vec=[val[1]]
    else:
      prod_embedding[i]=prod_embedding[i].replace('[tensor([',"")
      prod_embedding[i]=prod_embedding[i].replace(']',"")
      #prod_embedding[i]=int(prod_embedding[i])
      vec.append(prod_embedding[i])


In [ ]:
import tensorflow as tf
ogtensor=[]
for i in range(len(ogvec)):
  temp="".join(ogvec[i])
  v=temp.split(',')
  v=[float(x) for x in v]
  tensor=tf.convert_to_tensor(v, dtype=None, dtype_hint=None, name=None)
  ogvec[i]=v
  ogtensor.append(tensor)

In [ ]:
ogtensor[0]

Loss

In [ ]:
hinge_loss = nn.HingeEmbeddingLoss()
output = hinge_loss(input, target)
print(output)


tensor(1.0874, grad_fn=<MeanBackward0>)


In [ ]:
output.backward()

In [ ]:
input.grad

tensor([[0.0667, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0667, 0.0667, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]])